# 1. Instalaciones e Importaciones

In [1]:
# Instalar si es necesario (sklearn suele venir en Colab)
# !pip install scikit-learn nltk spacy pandas > /dev/null # Pandas para mostrar matrices fácil
# !python -m spacy download es_core_news_sm > /dev/null

# Importaciones
import nltk
import spacy
import re
import string
import pandas as pd # Para DataFrames

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [3]:
!python -m spacy download es_core_news_sm > /dev/null
nlp = spacy.load('es_core_news_sm')
print("Modelo de spaCy 'es_core_news_sm' cargado.")

Modelo de spaCy 'es_core_news_sm' cargado.


In [4]:
nltk.download('stopwords')
stopwords_es = nltk.corpus.stopwords.words('spanish')

print(f"\nUsando {len(stopwords_es)} stopwords de NLTK.")


Usando 313 stopwords de NLTK.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# 7. Micro-Laboratorio (Ejercicio Práctico)

**Consigna:**

Usando las `reviews` y las funciones de preprocesamiento de clases previas (o volviendo a procesarlas ahora):
1.  Asegúrate de tener la lista de `reviews_preprocesadas` (cada elemento es un string con los lemas unidos por espacios). Si no la tenés, generála usando la función `preprocesar_texto_para_vectorizar` sobre las `reviews` originales.
2.  **Crear Matriz BoW:**
    *   Instancia un `CountVectorizer`.
    *   Aplícalo a las `reviews_preprocesadas` usando `fit_transform()`.
    *   Obtén el vocabulario (`get_feature_names_out()`).
    *   Crea un DataFrame de Pandas para visualizar la matriz BoW.
3.  **Crear Matriz TF-IDF:**
    *   Instancia un `TfidfVectorizer`.
    *   Aplícalo a las **mismas** `reviews_preprocesadas` usando `fit_transform()`.
    *   **Importante:** Para comparar fácil, puedes pasarle el vocabulario aprendido por el CountVectorizer al TfidfVectorizer usando el argumento `vocabulary=`. O viceversa. La idea es que ambas matrices usen las mismas columnas en el mismo orden.
    *   Crea un DataFrame de Pandas para visualizar la matriz TF-IDF (redondea los valores a 3 decimales).
4.  **Analizar:**
    *   Imprime ambas matrices.
    *   Elige una o dos reviews. ¿Qué palabras tienen los pesos más altos en TF-IDF para esa review? ¿Coincide con lo que esperarías que sean las palabras clave de esa review?
    *   Busca alguna palabra que tenga un conteo > 0 en BoW pero un peso TF-IDF relativamente bajo. ¿Por qué podría ser? (Pista: ¿aparece en muchas reviews?).

In [5]:
# Dataset (el mismo del martes)
reviews = [
    "Una película emocionante con actuaciones brillantes. ¡Me encantó!",
    "Muy aburrida y lenta. El guión era predecible y los actores no convencían.",
    "Los efectos especiales fueron impresionantes, pero la historia dejaba mucho que desear.",
    "¡Qué gran comedia! Me reí sin parar durante toda la película.",
    "Un documental necesario que aborda temas importantes con profundidad y sensibilidad."
]

In [6]:
def preprocesar_texto_para_vectorizar(texto):
  # Limpieza básica
  texto = texto.lower()
  texto = re.sub(r'\d+', '', texto)
  texto = texto.translate(str.maketrans(string.punctuation + '¡¿', ' ' * len(string.punctuation + '¡¿')))
  texto = re.sub(r'\s+', ' ', texto).strip()
  # Lematización y filtrado con spaCy
  doc = nlp(texto)
  lemas = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]
  # Unir lemas en un solo string
  return " ".join(lemas)

In [7]:
reviews_procesadas = [preprocesar_texto_para_vectorizar(review) for review in reviews]

In [8]:
reviews_procesadas

['película emocionante actuación brillante encantar',
 'aburrido lento guión predecible actor convencer',
 'efecto especial impresionante historia dejar desear',
 'comedia reí parar película',
 'documental necesario abordar tema importante profundidad sensibilidad']

In [9]:
vectorizer_bow = CountVectorizer()
bow_matrix = vectorizer_bow.fit_transform(reviews_procesadas)

In [10]:
vocabulario = vectorizer_bow.get_feature_names_out()
print("Vocabulario aprendido:")
print(vocabulario)
print(f"\nTamaño del vocabulario: {len(vocabulario)}")

Vocabulario aprendido:
['abordar' 'aburrido' 'actor' 'actuación' 'brillante' 'comedia'
 'convencer' 'dejar' 'desear' 'documental' 'efecto' 'emocionante'
 'encantar' 'especial' 'guión' 'historia' 'importante' 'impresionante'
 'lento' 'necesario' 'parar' 'película' 'predecible' 'profundidad' 'reí'
 'sensibilidad' 'tema']

Tamaño del vocabulario: 27


In [11]:
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=vocabulario, index=[f"Doc_{i+1}" for i in range(len(reviews))])

In [12]:
vectorizer_tfidf = TfidfVectorizer()

In [13]:
tfidf_matrix = vectorizer_tfidf.fit_transform(reviews_procesadas)

In [15]:
vocabulario_tfidf = vectorizer_tfidf.get_feature_names_out()
vectorizer_bow_for_compare = CountVectorizer(vocabulary=vocabulario_tfidf)
bow_matrix_for_compare = vectorizer_bow_for_compare.fit_transform(reviews_procesadas)
bow_df_for_compare = pd.DataFrame(bow_matrix_for_compare.toarray(), columns=vocabulario_tfidf, index=[f"Doc_{i+1}" for i in range(len(reviews))])

In [16]:
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vocabulario_tfidf, index=[f"Doc_{i+1}" for i in range(len(reviews))])

In [17]:
print("--- Comparación BoW vs TF-IDF ---")

print("\nMatriz Bag-of-Words (Revisada con vocabulario TF-IDF):")
print(bow_df_for_compare)

print("\nMatriz TF-IDF:")
print(tfidf_df.round(3))

--- Comparación BoW vs TF-IDF ---

Matriz Bag-of-Words (Revisada con vocabulario TF-IDF):
       abordar  aburrido  actor  actuación  brillante  comedia  convencer  \
Doc_1        0         0      0          1          1        0          0   
Doc_2        0         1      1          0          0        0          1   
Doc_3        0         0      0          0          0        0          0   
Doc_4        0         0      0          0          0        1          0   
Doc_5        1         0      0          0          0        0          0   

       dejar  desear  documental  ...  impresionante  lento  necesario  parar  \
Doc_1      0       0           0  ...              0      0          0      0   
Doc_2      0       0           0  ...              0      1          0      0   
Doc_3      1       1           0  ...              1      0          0      0   
Doc_4      0       0           0  ...              0      0          0      1   
Doc_5      0       0           1  ...     

In [24]:
review1 = reviews[0]
review2 = reviews[1]

pesos_review1 = tfidf_df.loc['Doc_1']
pesos_review2 = tfidf_df.loc['Doc_2']

pesos_review1 = pesos_review1.sort_values(ascending=False)
pesos_review2 = pesos_review2.sort_values(ascending=False)

print(f"\nPesos más altos en la review 1:")
display(pesos_review1[pesos_review1 > 0])

print(f"\nPesos más altos en la review 2:")
display(pesos_review2[pesos_review2 > 0])


Pesos más altos en la review 1:


,Doc_1
actuación,0.463693
brillante,0.463693
emocionante,0.463693
encantar,0.463693
película,0.374105



Pesos más altos en la review 2:


,Doc_2
aburrido,0.408248
actor,0.408248
convencer,0.408248
lento,0.408248
predecible,0.408248
guión,0.408248


Si, estas son las palabras clave de cada review y los pesos tienen sentido.